# Just an example.You can alter sample code anywhere.

## mount your google drive

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

# You need to modify this part to the directory where your code is located
%cd "/content/drive/MyDrive/DL_Lab1"


/content/drive/MyDrive/DL_Lab1


## Import packages


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import model

In [5]:
#Fix the random seed
np.random.seed(0)

## Load the data and label

In [6]:
train_load = np.loadtxt('./data/kmnist-train.csv',delimiter=',',dtype="int")
test_load = np.loadtxt('./data/kmnist-test.csv',delimiter=',',dtype="int")

train_data=train_load[:,1:]
train_label=train_load[:,0]
test_data=test_load
print("shape of train_data: {}".format(train_data.shape))
print("shape of train_label: {}".format(train_label.shape))
print("shape of test_data: {}".format(test_data.shape))

shape of train_data: (60000, 784)
shape of train_label: (60000,)
shape of test_data: (10000, 784)


## Show the training data

In [7]:
#uncomment if you want to show the training data
#plt.figure(figsize=(20, 20))
#for index in range(10):
#    image = train_data[index+1000].reshape(28,28)
#    plt.subplot(2, 5, index+1)
#    plt.imshow(image)
#plt.show()

In [8]:
train_image_num = train_data.shape[0]
test_image_num = test_data.shape[0]
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)

print("train_image_num  is : {}".format(train_image_num))
print("test_image_num   is : {}".format(test_image_num))

train_image_num  is : 60000
test_image_num   is : 10000


## Validation image number

In [9]:
val_image_num=10000

## Convert the training labels to one hot vector

In [10]:
label_temp = np.zeros((train_image_num, 10), dtype = np.float32)
for i in range(train_image_num):
    label_temp[i][train_label[i]] = 1
train_label_onehot = np.copy(label_temp)
print("One-hot training labels shape:",train_label_onehot.shape)



One-hot training labels shape: (60000, 10)


## Hyperparameters

In [29]:
EPOCH = 75
Batch_size = 500 # 10000 should be divisible by batch_size
Learning_rate = 0.001

## Training

In [30]:
# please make sure you have place layer.py & network.py in 'model' folder
net = model.Network()

train_batch_num = (train_image_num  -  val_image_num  )//Batch_size
val_batch_num = (val_image_num)//Batch_size

for epoch in range(1, EPOCH+1):
    train_hit = 0
    val_hit = 0
    total_train_loss = 0.0
    total_val_loss = 0.0



    for it in range(train_batch_num):
        pred, train_loss = net.forward(train_data[it*Batch_size:(it+1)*Batch_size], train_label_onehot[it*Batch_size:(it+1)*Batch_size])
        pred_index = np.argmax(pred, axis=1)
        train_hit += (pred_index==train_label[it*Batch_size:(it+1)*Batch_size]).sum()
        total_train_loss += train_loss

        net.backward()
        net.update(Learning_rate)

    for titt in range(val_batch_num):
        tit=train_batch_num+titt
        pred, val_loss = net.forward(train_data[tit*Batch_size:(tit+1)*Batch_size], train_label_onehot[tit*Batch_size:(tit+1)*Batch_size])
        pred_index = np.argmax(pred, axis=1)
        val_hit += (pred_index==train_label[tit*Batch_size:(tit+1)*Batch_size]).sum()
        total_val_loss += val_loss

    print('Task1  | Epoch:%3d'%epoch, ' |Train Loss:%8.4f'%(total_train_loss/train_batch_num), ' |Train Acc:%3.4f'%(train_hit/(train_image_num-val_image_num)*100.0)
          , ' |Val Loss:%8.4f'%(total_val_loss/val_batch_num), ' |Val Acc:%3.4f'%(val_hit/val_image_num*100.0))

Task1  | Epoch:  1  |Train Loss:  2.3014  |Train Acc:14.2780  |Val Loss:  2.2997  |Val Acc:17.3700
Task1  | Epoch:  2  |Train Loss:  2.2981  |Train Acc:20.1980  |Val Loss:  2.2960  |Val Acc:22.7300
Task1  | Epoch:  3  |Train Loss:  2.2938  |Train Acc:24.1580  |Val Loss:  2.2908  |Val Acc:25.1200
Task1  | Epoch:  4  |Train Loss:  2.2876  |Train Acc:25.4340  |Val Loss:  2.2828  |Val Acc:26.4600
Task1  | Epoch:  5  |Train Loss:  2.2773  |Train Acc:26.5280  |Val Loss:  2.2688  |Val Acc:28.0700
Task1  | Epoch:  6  |Train Loss:  2.2580  |Train Acc:28.6020  |Val Loss:  2.2411  |Val Acc:30.5600
Task1  | Epoch:  7  |Train Loss:  2.2166  |Train Acc:31.9440  |Val Loss:  2.1776  |Val Acc:34.9500
Task1  | Epoch:  8  |Train Loss:  2.1156  |Train Acc:35.8040  |Val Loss:  2.0239  |Val Acc:36.9400
Task1  | Epoch:  9  |Train Loss:  1.9253  |Train Acc:36.5760  |Val Loss:  1.8119  |Val Acc:38.7300
Task1  | Epoch: 10  |Train Loss:  1.7263  |Train Acc:42.2640  |Val Loss:  1.6222  |Val Acc:48.4400
Task1  | E

## Dump for evaluation (upload your DL-test-predict.csv to kaggle )

In [20]:
test_pred_list = []
total_test = 0
for tit in range(test_image_num//Batch_size):
    pred, _ = net.forward(test_data[tit*Batch_size:(tit+1)*Batch_size], train_label_onehot[tit*Batch_size:(tit+1)*Batch_size])
    pred_index = np.argmax(pred, axis=1)
    test_pred_list += pred_index.tolist()
    total_test += Batch_size

print('Please make sure that total test images = 10000')
print(f'Total test images: {total_test} ')

print('Dump file...')
df = pd.DataFrame(test_pred_list, columns=["Category"])
df.to_csv('DL-test-predict.csv', index=True, index_label="Id")

Please make sure that total test images = 10000
Total test images: 10000 
Dump file...
